In [20]:
import pandas as pd
import numpy as np

btc = pd.read_csv("data/BTCUSD-2020-08-01.csv").drop("Unnamed: 0", axis=1)
eth = pd.read_csv("data/ETHUSD-2020-08-01.csv").drop("Unnamed: 0", axis=1)
eos = pd.read_csv("data/EOSUSD-2020-08-01.csv").drop("Unnamed: 0", axis=1)
xrp = pd.read_csv("data/XRPUSD-2020-08-01.csv").drop("Unnamed: 0", axis=1)

print(btc.head())


      timestamp  symbol  side  size    price  tickDirection  \
0  1.596326e+09  BTCUSD  Sell   250  11830.5  ZeroMinusTick   
1  1.596326e+09  BTCUSD  Sell   273  11830.5  ZeroMinusTick   
2  1.596326e+09  BTCUSD  Sell  1028  11830.5      MinusTick   
3  1.596326e+09  BTCUSD  Sell    90  11831.0      MinusTick   
4  1.596326e+09  BTCUSD   Buy     1  11831.5       PlusTick   

                             trdMatchID  grossValue  homeNotional  \
0  c99da727-003e-5567-8c66-aa58a192f5af   2113182.0           250   
1  012bf837-9d89-5a5b-ae0c-9fb284418e77   2307594.0           273   
2  10fe573c-5551-53ec-aedd-a671da182dec   8689404.0          1028   
3  9052a287-7e10-5b56-a4e5-9ccc22403abd    760713.0            90   
4  fdd09cbd-de02-505a-bcad-b350ba7c2acf      8452.0             1   

   foreignNotional  
0         0.021132  
1         0.023076  
2         0.086894  
3         0.007607  
4         0.000085  


In [21]:
def makeCandles(df, sec):
    # 参考: https://note.com/nagi7692/n/ne674d117d1b6?magazine_key=m0b2a506bf904
    df.drop(['tickDirection', 'trdMatchID', 'grossValue', 'homeNotional', 'foreignNotional'], axis=1, inplace=True)
    #86400本の秒足ができるように0秒に約定を入れる
    df = df.sort_index()
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit="s")
    df = df.rename(columns={'timestamp': 'exec_date'})
    df = df.set_index('exec_date')
    
    df['buy_size'] = df['size'].where(df['side'] == 'Buy', 0)
    df['buy_flag'] = df['side'] == 'Buy'
    df['sell_size'] = df['size'].where(df['side'] == 'Sell', 0)
    df['sell_flag'] = df['side'] == 'Sell'

    df_ohlcv = df.resample('{}S'.format(sec)).agg({"price": "ohlc", "size": "sum", "buy_size": "sum", "buy_flag": "sum",
                                      "sell_size": "sum", "sell_flag": "sum", })
    df_ohlcv.columns = ['open', 'high', 'low', 'close', 'volume', 'buy_vol', 'buy_num', 'sell_vol', 'sell_num']
    df_ohlcv['buy_num'] = df_ohlcv['buy_num'].astype(int)
    df_ohlcv['sell_num'] = df_ohlcv['sell_num'].astype(int)
    df_ohlcv.ffill(inplace=True)

    return df_ohlcv


In [22]:
btc = makeCandles(btc,1)
eth = makeCandles(eth,1)
eos = makeCandles(eos,1)
xrp = makeCandles(xrp,1)

In [23]:
btc_ret = np.log(btc["close"]) - np.log(btc["close"].shift(1))
eth_ret = np.log(eth["close"]) - np.log(eth["close"].shift(1))
eos_ret = np.log(eos["close"]) - np.log(eos["close"].shift(1))
xrp_ret = np.log(xrp["close"]) - np.log(xrp["close"].shift(1))

print(f"CoV_BTC: {np.std(btc_ret) / np.mean(btc_ret)}, ret_mean: {np.mean(btc_ret)}, ret_std: {np.std(btc_ret)} ")
print(f"CoV_ETH: {np.std(eth_ret) / np.mean(eth_ret)}, ret_mean: {np.mean(eth_ret)}, ret_std: {np.std(eth_ret)}  ")
print(f"CoV_EOS: {np.std(eos_ret) / np.mean(eos_ret)}, ret_mean: {np.mean(eos_ret)}, ret_std: {np.std(eos_ret)}  ")
print(f"CoV_XRP: {np.std(xrp_ret) / np.mean(xrp_ret)}, ret_mean: {np.mean(xrp_ret)}, ret_std: {np.std(xrp_ret)}  ")


CoV_BTC: 156.2332812450852, ret_mean: 4.692120804553059e-07, ret_std: 7.330654292936536e-05 
CoV_ETH: 109.51128673415009, ret_mean: 1.2885198338014512e-06, ret_std: 0.00014110746498207014  
CoV_EOS: 176.5669925932244, ret_mean: 7.795246622127942e-07, ret_std: 0.00013763832525916217  
CoV_XRP: 166.37810017875455, ret_mean: 1.3317309995502563e-06, ret_std: 0.0002215708736543255  
